#### Removing duplicates from the dataset

Having a lot of duplicates in the dataset will lead to bad models that are biased to the duplicate data

In [49]:
#Import the required modules
from imutils import paths
import numpy as np
import argparse
import cv2
import os

In [50]:
last_index = 30 # specify the length of the name of the class to compare

#### calculating the hash for all the images

In [51]:
#create and calculate the hash for each image
#if two images are similar then delete one
def dhash(image,hashSize=8):
  #convert the image to grayscale and resze the grayscale image
  #add a single column (width) so we can compute the horizontal gradient
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  resized = cv2.resize(gray, (hashSize+1, hashSize))
  
  #compute the relative horizontal gradient between adjacent column pixels
  diff = resized[:, 1:] > resized[:, :-1]
  
  #convert the difference image to a has and return it
  return sum([2 ** i for (i,v) in enumerate(diff.flatten()) if v])

#### Create a map where the key is the hash and the value is the image

In [52]:
DATASET_PATH = "../../dataset"
#grab the paths to all images in our input dataset directory
#initialise the hashes dictionary
print("[INFO] computing image hashes ....")
imagePaths = list(paths.list_images(DATASET_PATH))
hashes ={}



[INFO] computing image hashes ....


In [53]:
len(imagePaths)


1365

#Loop for finding all duplicates

* Calculating a map in which the key is the hash of the image
* Value is list of images with the same hash i.e. duplicate images

In [54]:
#loop over the image paths
for imagePath in imagePaths:
  #load the input image and compute the hash
  image =cv2.imread(imagePath)
  h = dhash(image)
  
  #grab all image paths with the hash, add the current image
  #path to it, and store the list back in the hashes dictionary
  p = hashes.get(h, [])
  p.append(imagePath)
  hashes[h] =p

In [55]:
count = 0
for (h, hashedPaths) in hashes.items():
  if (len(hashedPaths)>1):
    count += 1
    print(f'{hashedPaths[0].split(os.sep)[-1][0:last_index]} | {hashedPaths[1].split(os.sep)[-1][0:last_index]}')
print(len(hashes.items()))

car-bus_alltypes-1.jpg | car-bus_alltypes-203.jpg
car-bus_alltypes-10.jpg | car-bus_alltypes-123.jpg
car-bus_alltypes-103.jpg | car-bus_alltypes-20.jpg
car-bus_alltypes-104.jpg | car-bus_alltypes-32.jpg
car-bus_alltypes-11.jpg | car-bus_alltypes-215.jpg
car-bus_alltypes-111.jpg | car-bus_alltypes-64.jpg
car-bus_alltypes-113.jpg | car-bus_alltypes-176.jpg
car-bus_alltypes-115.jpg | car-bus_alltypes-69.jpg
car-bus_alltypes-117.jpg | car-bus_alltypes-16.jpg
car-bus_alltypes-120.jpg | car-bus_alltypes-71.jpg
car-bus_alltypes-121.jpg | car-bus_alltypes-51.jpg
car-bus_alltypes-125.jpg | car-bus_alltypes-224.jpg
car-bus_alltypes-126.jpg | car-bus_alltypes-191.jpg
car-bus_alltypes-13.jpg | car-bus_alltypes-147.jpg
car-bus_alltypes-134.jpg | car-bus_alltypes-188.jpg
car-bus_alltypes-136.jpg | car-bus_alltypes-25.jpg
car-bus_alltypes-137.jpg | car-bus_alltypes-81.jpg
car-bus_alltypes-139.jpg | car-bus_alltypes-62.jpg
car-bus_alltypes-14.jpg | car-bus_alltypes-38.jpg
car-bus_alltypes-145.jpg | ca